In [2]:
# Single Colab Cell - Sentiment Analysis System
# Just run this cell and click the ngrok URL!

!pip install flask flask-cors pyngrok -q

from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
from pyngrok import ngrok
import threading
import os

# Create Flask app
app = Flask(__name__)
CORS(app)

# HTML content
HTML_CONTENT = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Sentiment Analysis System</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <style>
        @keyframes spin { to { transform: rotate(360deg); } }
        .animate-spin { animation: spin 1s linear infinite; }
        .line-clamp-2 { display: -webkit-box; -webkit-line-clamp: 2; -webkit-box-orient: vertical; overflow: hidden; }
    </style>
</head>
<body class="bg-gradient-to-br from-blue-50 via-purple-50 to-pink-50 min-h-screen p-6">
    <div class="max-w-7xl mx-auto">
        <div class="text-center mb-8">
            <h1 class="text-4xl font-bold bg-gradient-to-r from-purple-600 to-pink-600 bg-clip-text text-transparent mb-3">
                🌟 Sentiment Analysis System
            </h1>
            <p class="text-gray-600 text-lg">Real-time text sentiment and emotion detection using NLP</p>
        </div>

        <div class="grid grid-cols-1 md:grid-cols-4 gap-4 mb-6">
            <div class="bg-white rounded-xl shadow-md p-4 border-l-4 border-blue-500">
                <p class="text-gray-500 text-sm">Total Analyzed</p>
                <p class="text-2xl font-bold text-gray-800" id="totalCount">0</p>
            </div>
            <div class="bg-white rounded-xl shadow-md p-4 border-l-4 border-green-500">
                <p class="text-gray-500 text-sm">Positive</p>
                <p class="text-2xl font-bold text-green-600" id="positiveCount">0</p>
            </div>
            <div class="bg-white rounded-xl shadow-md p-4 border-l-4 border-red-500">
                <p class="text-gray-500 text-sm">Negative</p>
                <p class="text-2xl font-bold text-red-600" id="negativeCount">0</p>
            </div>
            <div class="bg-white rounded-xl shadow-md p-4 border-l-4 border-gray-500">
                <p class="text-gray-500 text-sm">Neutral</p>
                <p class="text-2xl font-bold text-gray-600" id="neutralCount">0</p>
            </div>
        </div>

        <div class="grid grid-cols-1 lg:grid-cols-3 gap-6">
            <div class="lg:col-span-2">
                <div class="bg-white rounded-xl shadow-lg p-6 mb-4">
                    <h2 class="text-xl font-bold text-gray-800 mb-4">📄 Text Input</h2>
                    <textarea id="textInput" placeholder="Enter text to analyze sentiment..." class="w-full h-32 p-4 border-2 border-gray-200 rounded-lg focus:border-purple-500 focus:outline-none resize-none"></textarea>
                    <div class="flex items-center gap-2 mt-4 flex-wrap">
                        <button id="analyzeBtn" class="flex items-center gap-2 bg-gradient-to-r from-purple-600 to-pink-600 text-white px-6 py-2 rounded-lg hover:from-purple-700 hover:to-pink-700 transition-all">✉️ Analyze</button>
                        <button id="clearBtn" class="flex items-center gap-2 bg-gray-200 text-gray-700 px-6 py-2 rounded-lg hover:bg-gray-300 transition-all">🗑️ Clear</button>
                        <button id="exportBtn" class="flex items-center gap-2 bg-blue-600 text-white px-6 py-2 rounded-lg hover:bg-blue-700 transition-all ml-auto hidden">💾 Export</button>
                    </div>
                    <div class="mt-6">
                        <p class="text-sm text-gray-600 mb-2 font-medium">Quick Samples:</p>
                        <div class="flex flex-wrap gap-2" id="samplesContainer"></div>
                    </div>
                </div>
                <div id="resultsSection" class="bg-white rounded-xl shadow-lg p-6 hidden">
                    <h2 class="text-xl font-bold text-gray-800 mb-4">📊 Analysis Results</h2>
                    <div id="resultsContent"></div>
                </div>
            </div>
            <div class="lg:col-span-1">
                <div class="bg-white rounded-xl shadow-lg p-6 sticky top-6">
                    <h2 class="text-xl font-bold text-gray-800 mb-4">📈 Analysis History</h2>
                    <div id="historyContainer" class="space-y-3 max-h-[600px] overflow-y-auto">
                        <div class="text-center py-8 text-gray-400"><p>⚠️ No analysis yet</p></div>
                    </div>
                </div>
            </div>
        </div>
        <div class="mt-8 text-center text-sm text-gray-500">
            <p>Built with Python Flask • Lexicon-based NLP • Running on Google Colab</p>
        </div>
    </div>

    <script>
        const sampleTexts = [
            "I absolutely love this product! It exceeded all my expectations and the customer service was amazing!",
            "This is the worst experience I've ever had. Completely disappointed and frustrated.",
            "The product is okay. Nothing special but it works as described.",
            "I'm so happy and excited about this purchase! Best decision ever! 😊",
            "Terrible quality, waste of money. Very unhappy with this purchase."
        ];

        let stats = { totalAnalyzed: 0, positive: 0, negative: 0, neutral: 0 };
        let history = [];

        const positiveWords = ['good', 'great', 'excellent', 'amazing', 'wonderful', 'fantastic', 'love', 'loved', 'best', 'awesome', 'perfect', 'beautiful', 'happy', 'excited', 'joy', 'brilliant', 'outstanding', 'superb', 'pleased', 'delighted', 'satisfied', 'impressive', 'remarkable', 'incredible', 'exceptional', 'magnificent', 'terrific', 'fabulous', 'marvelous', 'splendid', 'enjoy'];
        const negativeWords = ['bad', 'terrible', 'awful', 'horrible', 'worst', 'hate', 'hated', 'poor', 'disappointing', 'disappointed', 'sad', 'angry', 'frustrated', 'annoyed', 'upset', 'pathetic', 'useless', 'waste', 'disgusting', 'appalling', 'dreadful', 'inferior', 'substandard', 'defective', 'faulty', 'broken', 'damaged', 'failed', 'failure', 'regret', 'unhappy'];

        const emotionWords = {
            joy: ['happy', 'joy', 'excited', 'delighted', 'cheerful', 'pleased', 'glad', 'enjoy'],
            sadness: ['sad', 'unhappy', 'disappointed', 'depressed', 'miserable', 'gloomy'],
            anger: ['angry', 'mad', 'furious', 'annoyed', 'irritated', 'frustrated', 'hate'],
            fear: ['afraid', 'scared', 'worried', 'anxious', 'nervous', 'terrified'],
            surprise: ['surprised', 'amazed', 'shocked', 'astonished', 'unexpected']
        };

        const emotionEmojis = { joy: '😊', sadness: '😢', anger: '😠', fear: '😨', surprise: '😮' };

        function preprocessText(text) {
            let processed = text.toLowerCase().replace(/[^\w\s]/g, ' ').replace(/\s+/g, ' ').trim();
            const stopwords = ['the', 'is', 'at', 'which', 'on', 'a', 'an', 'as', 'are', 'was', 'were', 'been', 'be', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should'];
            const words = processed.split(' ');
            const filtered = words.filter(word => !stopwords.includes(word) && word.length > 2);
            return { original: text, cleaned: processed, tokens: filtered, wordCount: words.length, tokenCount: filtered.length };
        }

        function analyzeSentiment(tokens) {
            let positiveScore = 0, negativeScore = 0;
            const detectedEmotions = {};

            tokens.forEach(token => {
                if (positiveWords.includes(token)) positiveScore++;
                if (negativeWords.includes(token)) negativeScore++;
                Object.keys(emotionWords).forEach(emotion => {
                    if (emotionWords[emotion].includes(token)) {
                        detectedEmotions[emotion] = (detectedEmotions[emotion] || 0) + 1;
                    }
                });
            });

            const total = positiveScore + negativeScore;
            let sentiment, confidence;

            if (total === 0) {
                sentiment = 'neutral'; confidence = 0.5;
            } else {
                const positiveRatio = positiveScore / total;
                if (positiveRatio > 0.6) {
                    sentiment = 'positive';
                    confidence = Math.min(0.6 + (positiveRatio - 0.6) * 1.5, 0.98);
                } else if (positiveRatio < 0.4) {
                    sentiment = 'negative';
                    confidence = Math.min(0.6 + (0.4 - positiveRatio) * 1.5, 0.98);
                } else {
                    sentiment = 'neutral';
                    confidence = 0.5 + Math.abs(positiveRatio - 0.5);
                }
            }

            const emotionScores = {};
            const totalEmotions = Object.values(detectedEmotions).reduce((a, b) => a + b, 0);
            if (totalEmotions > 0) {
                Object.keys(detectedEmotions).forEach(emotion => {
                    emotionScores[emotion] = Math.min((detectedEmotions[emotion] / totalEmotions) * 100, 95);
                });
            }

            return {
                sentiment, confidence: confidence * 100, positiveScore, negativeScore, emotions: emotionScores,
                keywords: { positive: tokens.filter(t => positiveWords.includes(t)), negative: tokens.filter(t => negativeWords.includes(t)) }
            };
        }

        function displayResults(result) {
            const section = document.getElementById('resultsSection');
            const content = document.getElementById('resultsContent');
            const sentimentColors = { positive: 'text-green-600 bg-green-50 border-green-200', negative: 'text-red-600 bg-red-50 border-red-200', neutral: 'text-gray-600 bg-gray-50 border-gray-200' };
            const sentimentIcons = { positive: '😊', negative: '😢', neutral: '😐' };

            let html = `
                <div class="border-2 rounded-lg p-4 mb-4 ${sentimentColors[result.analysis.sentiment]}">
                    <div class="flex items-center justify-between mb-2">
                        <div class="flex items-center gap-3">
                            <span class="text-3xl">${sentimentIcons[result.analysis.sentiment]}</span>
                            <div>
                                <p class="text-sm font-medium uppercase tracking-wide">Sentiment</p>
                                <p class="text-2xl font-bold capitalize">${result.analysis.sentiment}</p>
                            </div>
                        </div>
                        <div class="text-right">
                            <p class="text-sm font-medium">Confidence</p>
                            <p class="text-2xl font-bold">${result.analysis.confidence.toFixed(1)}%</p>
                        </div>
                    </div>
                    <div class="w-full bg-white bg-opacity-50 rounded-full h-2">
                        <div class="h-2 rounded-full ${result.analysis.sentiment === 'positive' ? 'bg-green-600' : result.analysis.sentiment === 'negative' ? 'bg-red-600' : 'bg-gray-600'}" style="width: ${result.analysis.confidence}%"></div>
                    </div>
                </div>
                <div class="bg-blue-50 border border-blue-200 rounded-lg p-4 mb-4">
                    <h3 class="font-semibold text-blue-900 mb-2">Preprocessing Statistics</h3>
                    <div class="grid grid-cols-2 gap-2 text-sm">
                        <div><span class="text-blue-700">Original Words:</span><span class="font-bold ml-2">${result.preprocessed.wordCount}</span></div>
                        <div><span class="text-blue-700">Processed Tokens:</span><span class="font-bold ml-2">${result.preprocessed.tokenCount}</span></div>
                        <div><span class="text-blue-700">Positive Signals:</span><span class="font-bold ml-2 text-green-600">${result.analysis.positiveScore}</span></div>
                        <div><span class="text-blue-700">Negative Signals:</span><span class="font-bold ml-2 text-red-600">${result.analysis.negativeScore}</span></div>
                    </div>
                </div>`;

            if (Object.keys(result.analysis.emotions).length > 0) {
                html += `<div class="bg-purple-50 border border-purple-200 rounded-lg p-4 mb-4"><h3 class="font-semibold text-purple-900 mb-3">Emotions Detected</h3><div class="space-y-2">`;
                Object.entries(result.analysis.emotions).forEach(([emotion, score]) => {
                    html += `<div class="flex items-center gap-2"><span class="text-2xl">${emotionEmojis[emotion]}</span><div class="flex-1"><div class="flex justify-between mb-1"><span class="text-sm font-medium capitalize text-purple-900">${emotion}</span><span class="text-sm font-bold text-purple-700">${score.toFixed(0)}%</span></div><div class="w-full bg-purple-200 rounded-full h-2"><div class="bg-purple-600 h-2 rounded-full" style="width: ${score}%"></div></div></div></div>`;
                });
                html += `</div></div>`;
            }

            if (result.analysis.keywords.positive.length > 0 || result.analysis.keywords.negative.length > 0) {
                html += `<div class="grid grid-cols-2 gap-4">`;
                if (result.analysis.keywords.positive.length > 0) {
                    html += `<div class="bg-green-50 border border-green-200 rounded-lg p-3"><h3 class="font-semibold text-green-900 mb-2 text-sm">Positive Keywords</h3><div class="flex flex-wrap gap-1">${result.analysis.keywords.positive.map(word => `<span class="text-xs bg-green-200 text-green-800 px-2 py-1 rounded">${word}</span>`).join('')}</div></div>`;
                }
                if (result.analysis.keywords.negative.length > 0) {
                    html += `<div class="bg-red-50 border border-red-200 rounded-lg p-3"><h3 class="font-semibold text-red-900 mb-2 text-sm">Negative Keywords</h3><div class="flex flex-wrap gap-1">${result.analysis.keywords.negative.map(word => `<span class="text-xs bg-red-200 text-red-800 px-2 py-1 rounded">${word}</span>`).join('')}</div></div>`;
                }
                html += `</div>`;
            }

            content.innerHTML = html;
            section.classList.remove('hidden');
        }

        function updateHistory() {
            const container = document.getElementById('historyContainer');
            if (history.length === 0) {
                container.innerHTML = '<div class="text-center py-8 text-gray-400"><p>⚠️ No analysis yet</p></div>';
                return;
            }
            const sentimentColors = { positive: 'border-green-300 bg-green-50', negative: 'border-red-300 bg-red-50', neutral: 'border-gray-300 bg-gray-50' };
            container.innerHTML = history.map((item, index) => `
                <div class="border-2 ${sentimentColors[item.analysis.sentiment]} rounded-lg p-3 hover:shadow-md transition-all cursor-pointer" onclick="showHistoryItem(${index})">
                    <div class="flex items-start justify-between mb-2">
                        <div class="px-2 py-1 rounded text-xs font-bold ${item.analysis.sentiment === 'positive' ? 'bg-green-600 text-white' : item.analysis.sentiment === 'negative' ? 'bg-red-600 text-white' : 'bg-gray-600 text-white'}">${item.analysis.sentiment.toUpperCase()}</div>
                        <span class="text-xs text-gray-500">${item.timestamp}</span>
                    </div>
                    <p class="text-sm text-gray-700 line-clamp-2">${item.text}</p>
                    <div class="flex items-center gap-2 mt-2">
                        <div class="text-xs text-gray-500">${item.analysis.confidence.toFixed(0)}% confident</div>
                        ${Object.keys(item.analysis.emotions).length > 0 ? `<div class="text-xs flex gap-1">${Object.keys(item.analysis.emotions).slice(0, 2).map(e => emotionEmojis[e]).join('')}</div>` : ''}
                    </div>
                </div>
            `).join('');
            document.getElementById('exportBtn').classList.remove('hidden');
        }

        function updateStats() {
            document.getElementById('totalCount').textContent = stats.totalAnalyzed;
            document.getElementById('positiveCount').textContent = stats.positive;
            document.getElementById('negativeCount').textContent = stats.negative;
            document.getElementById('neutralCount').textContent = stats.neutral;
        }

        function showHistoryItem(index) {
            displayResults(history[index]);
            window.scrollTo({ top: 0, behavior: 'smooth' });
        }

        function analyzeText() {
            const text = document.getElementById('textInput').value.trim();
            if (!text) return;
            const btn = document.getElementById('analyzeBtn');
            btn.innerHTML = '<div class="w-4 h-4 border-2 border-white border-t-transparent rounded-full animate-spin"></div> Analyzing...';
            btn.disabled = true;

            setTimeout(() => {
                const preprocessed = preprocessText(text);
                const analysis = analyzeSentiment(preprocessed.tokens);
                const result = { id: Date.now(), timestamp: new Date().toLocaleString(), text: text, preprocessed: preprocessed, analysis: analysis };
                displayResults(result);
                history.unshift(result);
                if (history.length > 10) history.pop();
                stats.totalAnalyzed++;
                stats[analysis.sentiment]++;
                updateStats();
                updateHistory();
                btn.innerHTML = '✉️ Analyze';
                btn.disabled = false;
            }, 1000);
        }

        function exportResults() {
            const data = { statistics: stats, history: history, exportDate: new Date().toLocaleString() };
            const blob = new Blob([JSON.stringify(data, null, 2)], { type: 'application/json' });
            const url = URL.createObjectURL(blob);
            const a = document.createElement('a');
            a.href = url;
            a.download = `sentiment_analysis_${Date.now()}.json`;
            a.click();
        }

        document.addEventListener('DOMContentLoaded', () => {
            const samplesContainer = document.getElementById('samplesContainer');
            sampleTexts.forEach((sample, idx) => {
                const btn = document.createElement('button');
                btn.className = 'text-xs bg-purple-100 text-purple-700 px-3 py-1 rounded-full hover:bg-purple-200 transition-all';
                btn.textContent = `Sample ${idx + 1}`;
                btn.onclick = () => { document.getElementById('textInput').value = sample; };
                samplesContainer.appendChild(btn);
            });
            document.getElementById('analyzeBtn').onclick = analyzeText;
            document.getElementById('clearBtn').onclick = () => { document.getElementById('textInput').value = ''; };
            document.getElementById('exportBtn').onclick = exportResults;
            document.getElementById('textInput').addEventListener('keypress', (e) => { if (e.key === 'Enter' && e.ctrlKey) analyzeText(); });
        });
    </script>
</body>
</html>
'''

@app.route('/')
def home():
    return HTML_CONTENT

# Start Flask in background thread
def run_flask():
    app.run(port=5000)

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print('🚀 Sentiment Analysis System is LIVE!')
print(f'🌐 Click here: {public_url}')
print('\n✅ Features:')
print('   - Real-time sentiment analysis')
print('   - Emotion detection')
print('   - Statistics dashboard')
print('   - Analysis history')
print('   - Export results')
print('\n⚠️ Keep this cell running to use the app!')

# Start server
threading.Thread(target=run_flask, daemon=True).start()

# Keep running
import time
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print('\n🛑 Server stopped')
```

---

## 🎯 **How to Use:**

1. **Copy the entire code above**
2. **Paste in a new Colab cell**
3. **Click Run (▶️)**
4. **Wait 10-15 seconds** for installation
5. **Click the ngrok URL** that appears (looks like: `https://xxxx-xx-xx.ngrok-free.app`)
6. **Your app is LIVE!** 🎉

---

## ✅ **What You'll See:**
```
🚀 Sentiment Analysis System is LIVE!
🌐 Click here: https://1234-xx-xx-xx-xx.ngrok-free.app

✅ Features:
   - Real-time sentiment analysis
   - Emotion detection
   - Statistics dashboard
   - Analysis history
   - Export results

⚠️ Keep this cell running to use the app!

<>:119: SyntaxWarning: invalid escape sequence '\w'
<>:119: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipython-input-786559335.py:119: SyntaxWarning: invalid escape sequence '\w'
  let processed = text.toLowerCase().replace(/[^\w\s]/g, ' ').replace(/\s+/g, ' ').trim();


SyntaxError: invalid character '▶' (U+25B6) (ipython-input-786559335.py, line 356)